In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder


from tqdm.notebook import tqdm

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, roc_curve
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier

# ustawia domyślną wielkość wykresów
plt.rcParams['figure.figsize'] = (12,8)
# to samo tylko dla tekstu
plt.rcParams['font.size'] = 16

/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train = pd.read_csv('data/car_prices_poland/Car_Prices_Poland.csv')

## Obróbka danych

In [3]:
train.isnull().sum()

Unnamed: 0             0
mark                   0
model                  0
generation_name    30085
year                   0
mileage                0
vol_engine             0
fuel                   0
city                   0
province               0
price                  0
dtype: int64

In [4]:
train = train.drop(columns=['generation_name', 'Unnamed: 0'])

In [5]:
ord_enc = OrdinalEncoder()

train["mark"] = ord_enc.fit_transform(train[["mark"]])
train["model"] = ord_enc.fit_transform(train[["model"]])
train["fuel"] = ord_enc.fit_transform(train[["fuel"]])
train["province"] = ord_enc.fit_transform(train[["province"]])
train["city"] = ord_enc.fit_transform(train[["city"]])


## Podział danych

In [6]:
y = train['price']
X = train.drop("price", axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=420, train_size=0.66)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=420, train_size=0.66)

In [7]:
X_train

,mark,model,year,mileage,vol_engine,fuel,city,province
61271,11.0,109.0,2020,26553,1950,1.0,4290.0,20.0
68673,17.0,249.0,2010,259985,1900,1.0,425.0,3.0
103703,14.0,200.0,2013,150000,1386,3.0,3770.0,6.0
22973,1.0,226.0,2022,10,2967,1.0,2882.0,6.0
70720,20.0,51.0,2006,373000,2231,1.0,4154.0,20.0
...,...,...,...,...,...,...,...,...
117363,22.0,321.0,2020,25600,1969,1.0,2426.0,16.0
62239,11.0,181.0,2007,253000,3996,1.0,3819.0,20.0
21055,1.0,32.0,2012,143000,2967,1.0,3693.0,18.0
62536,11.0,306.0,2019,54800,1950,1.0,3132.0,6.0


In [8]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

print(X_val.shape)
print(y_val.shape)

(77831, 8)
(77831,)
(26463, 8)
(26463,)
(13633, 8)
(13633,)


## Random Forest

In [9]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train).score(X_test, y_test)

0.9381897017104386

In [11]:
#przetestujmy jak działa to dla różnych parametrów
scores = []
for i in [1, 2, 4, 8, 16, 32, 64, 128]:
    for j in  [1, 2, 4, 8, 16, 32, 64, 128]:
        for k in  [2, 4, 8, 16, 32, 64, 128]:
            best = [0,0,0,0]
            rf = RandomForestRegressor(max_depth = i, n_estimators = j, max_leaf_nodes = k)
            score = rf.fit(X_train, y_train).score(X_test, y_test)
            print (f"max_depth: {i},  n_estimators: {j}, max_leaf_nodes: {k}, score: {score}")
            if (score > best[3]):
                best = [i, j, k, score]
            scores.append(score)

max_depth: 1,  n_estimators: 1, max_leaf_nodes: 2, score: 0.3504403036400191
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, score: 0.35042167128185775
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 8, score: 0.3503335017984971
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, score: 0.35027409115119956
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 32, score: 0.3504295759388255
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 64, score: 0.3503307781524121
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 128, score: 0.3504312588248477
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 2, score: 0.35044532245896687
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 4, score: 0.3503700065646441
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 8, score: 0.3504232680012739
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 16, score: 0.35043591443473454
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 32, score: 0.35044795896915826
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 64, score: 0.350

max_depth: 2,  n_estimators: 128, max_leaf_nodes: 2, score: 0.35254566181327995
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 4, score: 0.6155426122740668
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 8, score: 0.6180770237931956
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 16, score: 0.6196943064570675
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 32, score: 0.6180471481262227
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 64, score: 0.6197037860172789
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 128, score: 0.61638854450522
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 2, score: 0.3503850681717209
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 4, score: 0.6137485618007595
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 8, score: 0.737876181500168
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 16, score: 0.7902089976057816
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 32, score: 0.789346610925139
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 64, score:

max_depth: 8,  n_estimators: 64, max_leaf_nodes: 2, score: 0.35256231008884586
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 4, score: 0.6269160881603986
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 8, score: 0.7433732784359147
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 16, score: 0.7986975727337312
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 32, score: 0.844021041723636
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 64, score: 0.8687683524819803
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 128, score: 0.8877731723233707
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 2, score: 0.3575373449989617
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 4, score: 0.6188749929258696
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 8, score: 0.7405242097812286
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 16, score: 0.7979278307122595
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 32, score: 0.8422259963824668
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 64

max_depth: 32,  n_estimators: 16, max_leaf_nodes: 128, score: 0.8941063123067644
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 2, score: 0.3546151584680878
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 4, score: 0.6137174779950112
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 8, score: 0.7420703774985165
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 16, score: 0.7947681431810912
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 32, score: 0.8423267376154933
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 64, score: 0.8683840613308661
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 128, score: 0.89472514150123
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 2, score: 0.3525264370269573
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 4, score: 0.6299188807020659
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 8, score: 0.7453934344299806
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 16, score: 0.7970490468409809
max_depth: 32,  n_estimators: 64, max_leaf_nod

max_depth: 128,  n_estimators: 8, max_leaf_nodes: 64, score: 0.8723533602789901
max_depth: 128,  n_estimators: 8, max_leaf_nodes: 128, score: 0.8940536687686556
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 2, score: 0.35042952972644825
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 4, score: 0.6268514544356607
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 8, score: 0.7510293708774519
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 16, score: 0.7970016567106781
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 32, score: 0.8415103422342819
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 64, score: 0.870377432913382
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 128, score: 0.8946451694527502
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 2, score: 0.3654339121302518
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 4, score: 0.6137565111511298
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 8, score: 0.7430191537651085
max_depth: 128,  n_estimators: 32,

In [14]:
print (f"mediana: {np.median(scores)}")
print (f"srednia: {np.mean(scores)}")
print (f"najlepszy: max_depth: {best[0]}, n_estimators: {best[1]}, max_leaf_nodes: {best[2]}, score: {best[3]}")

mediana: 0.7386716056238734
srednia: 0.6608425562228825
najlepszy: max_depth: 128, n_estimators: 128, max_leaf_nodes: 128, score: 0.8962583077113507


## XGBoost

In [15]:
xgb_reg = xgb.XGBRegressor(objective ='reg:linear', n_estimators = 100, seed = 123)
xgb_reg.fit(X_train, y_train, verbose=True, early_stopping_rounds=100, eval_metric="rmse", eval_set=[(X_val, y_val)])
xgb_reg.score(X_test, y_test)

[21:08:22] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:78978.29688
[1]	validation_0-rmse:59608.76953
[2]	validation_0-rmse:46810.71875


/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[3]	validation_0-rmse:38731.90234
[4]	validation_0-rmse:33747.10938
[5]	validation_0-rmse:30314.96484
[6]	validation_0-rmse:28424.15625
[7]	validation_0-rmse:27180.24609
[8]	validation_0-rmse:26231.63672
[9]	validation_0-rmse:25595.12500
[10]	validation_0-rmse:25271.23242
[11]	validation_0-rmse:25030.88867
[12]	validation_0-rmse:24915.65234
[13]	validation_0-rmse:24529.78711
[14]	validation_0-rmse:24393.43164
[15]	validation_0-rmse:24330.82617
[16]	validation_0-rmse:24275.04688
[17]	validation_0-rmse:23980.49414
[18]	validation_0-rmse:23853.24023
[19]	validation_0-rmse:23647.44336
[20]	validation_0-rmse:23517.18945
[21]	validation_0-rmse:23445.60938
[22]	validation_0-rmse:23114.34375
[23]	validation_0-rmse:22995.69727
[24]	validation_0-rmse:22901.15430
[25]	validation_0-rmse:22852.85156
[26]	validation_0-rmse:22843.57031
[27]	validation_0-rmse:22747.86133
[28]	validation_0-rmse:22629.48828
[29]	validation_0-rmse:22578.02539
[30]	validation_0-rmse:22558.84375
[31]	validation_0-rmse:2248

0.9380997897184561

In [ ]:
#przetestujmy jak działa to dla różnych parametrów
scores = []
for i in [1, 4, 16, 64, 128]:
    for j in  [1, 4, 16, 64, 128]:
        for k in  [4, 16, 64, 128]:
            for l in [1, 4, 16, 64, 128]:
                best = [0,0,0,0,0]
                xgb_reg = xgb.XGBRegressor(objective="reg:linear", 
                                       seed = 42, 
                                       use_label_encoder=False, 
                                       max_depth = i, 
                                       n_estimators = j, 
                                       max_leaf_nodes = k, 
                                       max_leaves = l)

                score = xgb_reg.fit(X_train, y_train, 
                                    verbose=False, 
                                    early_stopping_rounds=10, 
                                    eval_metric="rmse", eval_set=[(X_val, y_val)]).score(X_test, y_test)
                                                                  
                print (f"max_depth: {i},  n_estimators: {j}, max_leaf_nodes: {k}, max_leaves: {l}, score: {score}")
                if (score > best[4]):
                    best = [i, j, k, l, score]
                scores.append(score)

[22:58:55] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[22:58:55] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 1, score: -0.15225544970847582
[22:58:55] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[22:58:55] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wro

/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pan

max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 64, score: -0.15225544970847582
[22:58:55] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[22:58:55] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 128, score: -0.15225544970847582
[22:58:55] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[22:58:55] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistak

/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pan

[22:58:55] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[22:58:55] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, max_leaves: 16, score: -0.15225544970847582
[22:58:55] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[22:58:55] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged w

...

/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


max_depth: 128,  n_estimators: 128, max_leaf_nodes: 16, max_leaves: 16, score: 0.9362602048383744
[23:43:31] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:31] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


max_depth: 128,  n_estimators: 128, max_leaf_nodes: 16, max_leaves: 64, score: 0.9362602048383744
[23:43:51] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:51] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [ ]:
print (f"mediana: {np.median(scores)}")
print (f"srednia: {np.mean(scores)}")
print (f"najlepszy: max_depth: {best[0]}, n_estimators: {best[1]}, max_leaf_nodes: {best[2]}, max_leaves: {best[3]}, score: {best[4]}")

## Wnioski 
* Z XGBoost jest lepszy z parametrami domyślnymi niż Random Forest (także z domyślnymi).

* XGBoost okazuje się być średnio lepszy niż Random Forest.

* W przypadku obu modeli dobranie większych parametrów pozytywnie wpływa na score (wynik best)